## Crystal Structure Prediction Workflow Using Open Source Tools
### Generate AMOEBA Parameters Using PolType
Starting from an SDF file (e.g. from PubChem), the PolType program can be used to generate AMOEBA polarizable atomic multipole parameters for organic molecules. In the near future, we will be adding support for PolType (and its dependencies) to the binder/Dockerfile. Here we supply PolType resutls for carmbazepine as an example (https://pubchem.ncbi.nlm.nih.gov/compound/Carbamazepine). 

For more information on PolType from the lab of Pengyu Ren at UT Austin, please visit the following webpage:
https://github.com/pren/poltype/tree/poltype2

### Convert a CCDC CIF File to XYZ Format Using the Force Field X CIFtoXYZ Command
To convert CCDC CIF files to a format compatible with AMOEBA molecular mechanics algorithms, Aaron Nessler has created a Force Field X command called "CIFtoXYZ". The input to this command is the CCDC CIF file, and the force field information produced by PolType. As an example, here we convert a carmbazepine file (space group P 21/c) to XYZ format.

In [ ]:
// Convert a CCDC CIF file to Force Field X XYZ format.
var command = "test.CIFtoXYZ"; // CIFtoXYZ command.
var cif = "CBMZPN01.cif";      // CCDC CIF to convert to XYZ format.
var xyz = "CBZ.xyz";           // The XYZ file provides force field information for the conversion process.
String[] args = {command, cif, xyz};
ffxScript(args);

### Minimize a Crystal Structure Using the Force Field X CrystalMin Command
Crystal structures can be minimized using the AMOEBA PolType parameters with the Force Field X CrystalMin command. The experimental carmbazepine structure (CBMZPN01.cif) in XYZ format from the previous CIFtoXYZ command (CBMZPN01.xyz) will be used as an example. We select a gradient convergence criteria "eps" of 1.0e-3 RMS kcal/mol/Angstroms and the "-c" flag to indicate both coordinates and unit cell parameters should be optimized.

In [ ]:
// Minimize a Crystal Structure
var command = "CrystalMin";     // CrystalMin command.
var tolerance = "--eps=1.0e-1"; // RMS Gradient convergence criteria (kcal/mol/A)
var coordinates = "-c";         // Flag to indicate coordinates and unit cell parameters should be optimized.
var molecule = "CBMZPN01.xyz";  // The input xyz file.
String[] args = {command, tolerance, coordinates, xyz};
ffxScript(args);

### Evaluate the Potential Energy of a Crystal Structure Using the Force Field X Energy Command
The previous minimization produced a minimum energy structure in the file CBMZPN01.xyz_2. The potential energy of the structure is reported using the Force Field X energy command.

In [ ]:
// Evaluate the Potential Energy of a Crystal Structure
var command = "Energy";          // Energy command.
var molecule = "CBMZPN01.xyz_2"; // The input xyz file.
String[] args = {command, molecule};
ffxScript(args);

### Prepare a Directory for a Polymorph Search Using the PrepareSpaceGroups Command
Using the CBZ.xyz structure as input, we can prepare a subdirectory to search another space group for low energy polymorphs. For this example, we will choose the "H-3" space group (--sg=H-3) and specify a starting density of 1.2 g/cc (-d=1.2). The unit cell parameters will be chosen randomly to satisfy the selected density. Finally, we will apply a random symmetry operator to the starting rigid body coordiantes of carmbazepine (--rsym=1.0). The output is placed in a new subdirectory "H-3", including a coordinate file "CBZ.xyz" and a properties file "CBZ.properties".

In [ ]:
// Prepare a Space Group for a Polymorph Search
var command = "PrepareSpaceGroups";  // Prepare space group command.
var spacegroup = "--sg=H-3";         // Select the H-3 space group.
var density = "-d=1.2"               // Select a density of 1.2 g/cc.
var orient = "--rsym=1.0"            // Randomly orient the molecule within the unit cell. 
var molecule = "CBZ.xyz";            // The input xyz file.
String[] args = {command, spacegroup, density, orient, molecule};
ffxScript(args);

### Perform a Thermodynamically Driven Polymoprh Search Using the Force Field X Thermodynamics Command
A polymoprh search must be performed using a high-performance cluster with a large number of CPU cores and/or nodes. However, in the context of a Jupyter notebook, we can still demonstrate the Force Field X command for a thermodynamically driven polymoprh search approach. Further details of this approach are described on the Force Field X website (https://ffx.biochem.uiowa.edu/examples-CrystalNPT.html).

In [ ]:
// Perform a Thermodynamically Driven Polymorph Search
var command = "Thermodynamics";   // Search command.
var state = "-l=0.0";             // Search state needs to start in vacuum (Lambda = 0.0).
var atoms = "--ac=ALL";           // All atoms are alchemical and undergo a vacuum to crystal phase transition.
var temperature = "-t=298.15";    // Perform the polymorph search at 298.15 Kelvin.
var pressure = "-p=1.0";          // Perform the polymoprh search at 1 atm.
var integrator = "-i=stochastic"; // Use stochastic dynamics (i.e. Langevin).
var nSteps = "-n=1000";           // Number of molecular dynamics time steps.
var timeStep = "-d=1.0";          // Time step in femtoseconds.
var report =  "-r=0.01";          // Report results every 0.1 picoseconds.
var orient = "--rsym=0";          // Randomly orient the molecule within the unit cell.
var unitCell = "--ruc=1.2";       // Apply random unit cell parameters to achieve the a density of 1.2 g/cc.
var equilibrate = "-Q=0";         // Number of molecular dynamics equilibration steps.
var optimize = "-o";              // Optimize and save low-energy snapshots.
var molecule = "H-3/CBZ.xyz";     // The input xyz file.
String[] args = {command, state, atoms, temperature, pressure, 
                 integrator, nSteps, timeStep, report, 
                 orient, unitCell, equilibrate, optimize, molecule};
ffxScript(args);

### Analysis Following the Polymorph Search
Once the polymoprh search is completed for each space group of interest, then the Force Field X CrystalMin command (described above) can be applied to all saved coordinate snapshot. Low energy snapshots can be prioritized for structural comparison using the PACCOM tool by Dr. Okimasa Okada and for further evaluation (re-ranking) using Quantum Chemistry tools such as Quantum Expresso (QE). Currently PACCOM and QE are not included in the Dockerfile used to create the Jupyter environment, but the process to add them is underway.